# CREAZIONE E SELEZIONE DEL DATASET

In [ ]:
%%capture
!pip install allensdk

## Ricerca della Sessione Ottimale

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

from pprint import pprint # stampa formattata

import os
from tqdm import tqdm  

from allensdk.core.brain_observatory_cache import BrainObservatoryCache

import os

In [ ]:
import gc # Per il garbage collector

# --- Parametri Richiesti ---
REQUIRED_STRUCTURES = ['VISp']
REQUIRED_SESSION_TYPES = ['three_session_A']
SEARCH_MODE = "max"
EYE_TRACKING = True
SESSION_FAILED = False

# Inizializza la cache
boc = BrainObservatoryCache()

# Carica la lista completa di sessioni
all_sessions = boc.get_ophys_experiments(
    require_eye_tracking=EYE_TRACKING,
    include_failed=SESSION_FAILED,
    session_types=REQUIRED_SESSION_TYPES,
    targeted_structures=REQUIRED_STRUCTURES    
)
print(f"Trovate {len(all_sessions)} sessioni 'Session A' totali con eye tracking.")

# Inizializziamo i contatori per una nuova ricerca
max_neuron_count = -1 # Partiamo da -1 per assicurarci che il primo risultato valido venga salvato
best_session_id = None
ophys_session_data = None # Caricheremo il data-object solo alla fine

print(f"Stato iniziale: Avvio nuova scansione completa.")

# Imposta la lista delle sessioni da controllare uguale alla lista completa
remaining_sessions = all_sessions
print(f"Avvio della scansione per {len(remaining_sessions)} sessioni totali...")

# --- SCANSIONE CON PULIZIA ---
for session in remaining_sessions:
    temp_session_data = None # Resetta per il loop
    nwb_file_path = None     # Resetta per il loop
    session_id = session['id']
            
    try:
        print(f"Tentativo con sessione ID: {session_id}...")
        
        # <<< MODIFICA: Definiamo il percorso del file cache >>>
        # L'SDK salva i file qui. Dovremo eliminarlo manualmente.
        nwb_file_path = f"/kaggle/working/brain_observatory/ophys_experiment_data/{session_id}.nwb"
        
        # Se il file esiste da un run precedente fallito, rimuovilo prima
        if os.path.exists(nwb_file_path):
            print(f"  > Rimuovo file NWB residuo: {nwb_file_path}")
            os.remove(nwb_file_path)

        # 1. Scarica i dati (questo è il passaggio che richiede spazio)
        temp_session_data = boc.get_ophys_experiment_data(ophys_experiment_id=session_id)
        
        # 2. Controlla i dati della pupilla (requisito fondamentale)
        temp_session_data.get_pupil_size()
        temp_session_data.get_pupil_location()
        print(f"  > Dati pupillari validi trovati.")
        
        # 3. Controlla il numero di neuroni
        num_neurons_list = temp_session_data.get_cell_specimen_ids()
        current_num_neurons = len(num_neurons_list)
        print(f"  > Numero neuroni: {current_num_neurons}")
        
        # 4. Logica di aggiornamento dinamica
        update = False
        if SEARCH_MODE == "max":
            if current_num_neurons > max_neuron_count:
                update = True
        elif SEARCH_MODE == "min":
            if current_num_neurons > 0 and current_num_neurons < max_neuron_count: 
                update = True
        
        if update:
            print(f"  > !!! Nuovo record {SEARCH_MODE.upper()}! ({current_num_neurons}). Salvo ID.")
            max_neuron_count = current_num_neurons
            best_session_id = session_id
            # NON salviamo l'oggetto ophys_session_data, solo il suo ID
        else:
            print(f"  > Non è un record. Scarto.")
            
    except Exception as e:
        # Se i dati della pupilla o altri metodi falliscono, scarta la sessione
        print(f"  > Sessione {session_id} scartata. Errore: {e}")
        
        # Se l'errore è lo spazio su disco, fermati e segnala
        if "No space left on device" in str(e):
            print("!!! ERRORE: Spazio su disco nuovamente esaurito. Interruzione forzata.")
            # Non continuare il loop, rilancia l'errore
            raise e
        
        continue # Passa alla sessione successiva
        
    finally:
        # <<< MODIFICA: PULIZIA DEL DISCO >>>
        # Questo blocco viene eseguito SEMPRE, sia in caso di successo che di errore
        
        # Rilascia l'oggetto dalla memoria
        if temp_session_data is not None:
            del temp_session_data
            
        # Elimina il file NWB dal disco per liberare spazio
        if nwb_file_path and os.path.exists(nwb_file_path):
            try:
                print(f"  > Pulizia file NWB: {nwb_file_path}")
                os.remove(nwb_file_path)
            except Exception as clean_e:
                print(f"  > ERRORE durante la pulizia del file {nwb_file_path}: {clean_e}")
        
        # Forza il garbage collector per liberare memoria
        gc.collect()


# --- Conclusione della Ricerca ---
if best_session_id is None:
    print("\nATTENZIONE: Nessuna sessione valida trovata")
else:
    print(f"\n--- SCANSIONE COMPLETATA ---")
    print(f"Modalità di ricerca: {SEARCH_MODE.upper()}")
    print(f"Sessione selezionata (ID): {best_session_id}")
    print(f"Numero di neuroni: {max_neuron_count}")
    
    # <<< MODIFICA: Caricamento finale >>>
    # Ora, e solo ora, scarichiamo l'oggetto ophys_session_data del vincitore
    try:
        print(f"Caricamento finale dei dati per la sessione {best_session_id}...")
        
        # Pulisci il file se esiste, per sicurezza (es. se era il primo della lista)
        final_nwb_path = f"/kaggle/working/brain_observatory/ophys_experiment_data/{best_session_id}.nwb"
        if os.path.exists(final_nwb_path):
             os.remove(final_nwb_path)
             
        ophys_session_data = boc.get_ophys_experiment_data(ophys_experiment_id=best_session_id)
        print("Dati della sessione migliore caricati con successo.")
    except Exception as e:
        print(f"ERRORE CRITICO: Impossibile caricare i dati della sessione migliore {best_session_id}. Errore: {e}")
        ophys_session_data = None # Assicura che lo script fallisca dopo

if ophys_session_data is None:
    print("\nATTENZIONE: Oggetto 'ophys_session_data' non caricato. Il resto dello script fallirà.")

In [ ]:
best_session_id = 501729039

In [ ]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import sys

# ID della con il max neuroni trovati
TARGET_SESSION_ID = best_session_id

# Inizializza la cache
boc = BrainObservatoryCache()

print(f"Caricamento della sessione ID: {TARGET_SESSION_ID}...")

try:
    # Carica direttamente i dati della sessione
    ophys_session_data = boc.get_ophys_experiment_data(ophys_experiment_id=TARGET_SESSION_ID)
    
    # Salviamo l'ID per le celle successive (come i nomi dei run W&B)
    best_session_id = TARGET_SESSION_ID
    
    print(f"Sessione {best_session_id} caricata con successo.")
    
    # Eseguiamo un controllo di validità
    ophys_session_data.get_pupil_size()
    ophys_session_data.get_pupil_location()
    print("Dati pupillari confermati")
    
except Exception as e:
    print(f"ERRORE: Impossibile caricare la sessione {TARGET_SESSION_ID}. Errore: {e}")
    # Fermiamo l'esecuzione se il caricamento fallisce
    sys.exit("Caricamento dati fallito.")

In [ ]:
# Visualizza gli stimoli
print(ophys_session_data.get_stimulus_epoch_table())

tabella (un DataFrame pandas) che riassume quali stimoli visivi sono stati mostrati al topo e in quali frame

* drifting_gratings: Barre in movimento


* natural_movie: un estratto di un film

* spontaneous: Periodi in cui al topo veniva mostrato uno schermo grigio, per misurare l'attività neurale "a riposo" o spontanea.


In questo modo sappiamo in quali frame (start e end) andare a cercare i dati che ci interessano (i natural_movie_one e natural_movie_three) e l'attività neurale corrispondente.

In [ ]:
pprint(ophys_session_data.get_metadata())
num_neurons = ophys_session_data.get_cell_specimen_ids()
print(f"\nnumero nueroni per la sessione: {len(num_neurons)}")

# Pre-elaborazione dei Dati di Input

## Processamento Video (Input Visivo)

**carichiamo i file video grezzi (i "template")**
**stampiamo le dimensioni (num frame, h frame in pixel, l frame in pixel)**

In [ ]:
stimulus_template_movie_one = ophys_session_data.get_stimulus_template('natural_movie_one')
stimulus_template_movie_three = ophys_session_data.get_stimulus_template('natural_movie_three')
print(f"Dimensioni del video 'natural_movie_one': {stimulus_template_movie_one.shape}")
print(f"Dimensioni del video 'natural_movie_three': {stimulus_template_movie_three.shape}")

**I video originali hanno frame di 304x608 pixel ==> troppo grandi per essere processati perchè richiederebbe un'enorme quantità di memoria GPU e tempo di calcolo.
Eseguiamo il downsampling (ricampionamento) per ridurre ogni frame alla dimensione molto più piccola di 36x64 pixel.**

In [ ]:
from scipy import ndimage

videos = [stimulus_template_movie_one, stimulus_template_movie_three]
target_height = 36
target_width = 64

downsampled_videos = []
for video in videos:
    resized_frames = []
    for frame in video:
        zoom_factors = (target_height / frame.shape[0], target_width / frame.shape[1])
        new_frame = ndimage.zoom(frame, zoom_factors, order=1) #order = 1 Specifica il metodo di interpolazione bilineare
        resized_frames.append(new_frame)
    
    downsampled_videos.append(np.array(resized_frames))

pprint(downsampled_videos[0].shape)
pprint(downsampled_videos[1].shape) 

In [ ]:
def segment_data_into_clips(items,pos_num_frames=0, target_frames=140) -> (list): 
    """prende un array di dati e lo segmenta in "clip" più piccoli tutti della stessa lunghezza."""
    total_frames = items.shape[pos_num_frames]
    num_items = total_frames // target_frames #scartiamo i frame rimanenti
    if len(items.shape) == 3: #(frame, altezza, larghezza).
        return [items[i*target_frames:(i+1)*target_frames, :, :] for i in range(num_items)]
    elif pos_num_frames==1: # le risposte neurali hanno 2 dimensioni (neuroni, frame).
        return [items[:, i*target_frames:(i+1)*target_frames] for i in range(num_items)]
    else:
        return [items[i*target_frames:(i+1)*target_frames, :] for i in range(num_items)]

clips_movie_one = segment_data_into_clips(downsampled_videos[0])
clips_movie_three = segment_data_into_clips(downsampled_videos[1])

print(f"Numero clip in natural_movie_'one': {len(clips_movie_one)}")
print(f"Numero clip in natural_movie_'three': {len(clips_movie_three)}")

**entrambe da 140 frame**

**sanity check per confermare il numero totale di frame che sono rimasti nel dataset dopo l'operazione di segmentazione effettuata.
Dovremmo avere: 6*140 = 840 e 25*140= 3500**

In [ ]:
total_frames = 0
for clip in clips_movie_one:
    total_frames += clip.shape[0]

print(f"frame totali video 'one' dopo la segmentazione: {total_frames}")

total_frames = 0
for clip in clips_movie_three:
    total_frames += clip.shape[0]

print(f"frame totali video'three' dopo la segmentazione: {total_frames}")

**effettuiamo un controllo visivo per mostrare l'aspetto di un singolo frame di input**

In [ ]:
plt.imshow(clips_movie_one[0][50], cmap='gray')

In [ ]:
plt.imshow(clips_movie_three[0][50], cmap='gray')

## Normalizzazione Video

**effettuiamo la normalizzazione dei valori dei pixel dei video.
I modelli "imparano" meglio quando i dati di input si trovano in un intervallo di valori piccolo e coerente (come da 0 a 1), piuttosto che in un intervallo ampio (come da 0 a 255).**

In [ ]:
def calculate_statistics(items):
    """
    Concatena tutti gli item in un unico array 1D
    Calcola le statistiche globali (media, std, min, max) per un insieme di clip.
    """
    
    all_values = np.concatenate([item.astype(np.float32).ravel() for item in items])
    mean = np.mean(all_values)
    std = np.std(all_values)
    min_val = np.min(all_values)
    max_val = np.max(all_values)

    return mean, std, min_val, max_val

def apply_min_max_normalization(items, min_val, max_val, eps=1e-8):
    """
    Applica la normalizzazione Min-Max (scala 0-1) a una lista di item.
    Utilizza i valori min e max globali calcolati da calculate_statistics.
    Formula: (x - min) / (max - min)
    """
    range_val = max_val - min_val
    range_safe = np.where(range_val > 0, range_val, eps) #'eps' previene la divisione per zero

    return [(item - min_val) / range_safe for item in items]

mean1, std1, min1, max1 = calculate_statistics(clips_movie_one)
mean3, std3, min3, max3 = calculate_statistics(clips_movie_three)

print("Video One -> Mean:", mean1, "Std:", std1, "Min:", min1, "Max:", max1)
print("Video Three -> Mean:", mean3, "Std:", std3, "Min:", min3, "Max:", max3)

normalized_one = apply_min_max_normalization(clips_movie_one, min1,max1)
normalized_three = apply_min_max_normalization(clips_movie_three, min3,max3)

**Rappresentano (luminosità,contrasto,gamma dinamica)
Gamma dinamica ==> pixel completamente neri (0) ; pixel completamente bianchi (255)**

## Processamento Dati Comportamentali (Pupilla e Corsa)

**Ora carichiamo i dati comportamentali ==> la posizione del centro della pupilla del topo per ogni singolo frame dell'intero esperimento.**

In [ ]:
timestamps, pupil_tracking_data = ophys_session_data.get_pupil_location()
print(pupil_tracking_data.shape)

* 115735: È il numero totale di fotogrammi per l'intera sessione.
* 2: Rappresenta le coordinate (x, y) del centro della pupilla per ciascuno di quei fotogrammi.


**i video natural_movie_one e natural_movie_three sono stati mostrati 10 volte durante l'esperimento. Tra una ripetizione e l'altra c'erano delle pause (gli "offset"). La funzione filter_data serve a estrarre solo i dati esatti di queste 10 ripetizioni, scartando le pause.**

In [ ]:
def extract_stimulus_trials(item, num_item_frames, offset, pos=None):
    filtered_list = []
    for i in range(10):
        start = i * (num_item_frames + offset)
        end = start + num_item_frames
        if pos is not None: # se pos non è None: la forma è [num_neurons, num_frames]
            filtered_list.append(item[:, start:end])
        else:
            filtered_list.append(item[start:end,:])

    if pos is not None:
        return np.concatenate(filtered_list, axis=1)
    
    return np.concatenate(filtered_list, axis=0)

pupil_data_movie_one = pupil_tracking_data[38751:47810,:] #38751 (start) e 47810 (end)
pupil_data_movie_one = extract_stimulus_trials(pupil_data_movie_one, 840, 66) #Tra una ripetizione e l'altra ci sono 66 frame di pausa che vanno ignorati"
print(pupil_data_movie_one.shape)

**La funzione ha estratto 10 ripetizioni.
Ogni ripetizione era lunga 840 frame.
Totale frame: 10 × 840 = 8400 frame.
Il 2 rappresenta le coordinate (x, y) della pupilla.
Ora pupil_location_one è un array che contiene esattamente i dati di tracciamento oculare per i soli frame in cui il topo stava guardando il "video one"**

**come visto nella tabella degli stimoli, natural_movie_three è stato mostrato in due blocchi separati durante l'esperimento**

In [ ]:
tmp_pupil_location_three = [pupil_tracking_data[19741:37846,:], pupil_tracking_data[75867:93967,:]]

pupil_data_movie_three = []
pupil_data_movie_three.append(extract_stimulus_trials(tmp_pupil_location_three[0], 3500, 121))
pupil_data_movie_three.append(extract_stimulus_trials(tmp_pupil_location_three[1], 3500, 120))

print(pupil_data_movie_three[0].shape)
print(pupil_data_movie_three[1].shape)

**5 ripetizioni × 3500 frame/ripetizione = 17.500 frame totali (con 2 coordinate x, y).**

In [ ]:
# Segmenta il blocco unico di "movie one" (che contiene 10 trial)
pupil_location_one_segmented = segment_data_into_clips(pupil_data_movie_one)

# Segmenta il primo blocco di "movie three" (5 trial)
pupil_location_three_segmented = segment_data_into_clips(pupil_data_movie_three[0])

# Estende la lista aggiungendo i segmenti del secondo blocco di "movie three" (altri 5 trial)
pupil_location_three_segmented.extend(segment_data_into_clips(pupil_data_movie_three[1]))

print(len(pupil_location_one_segmented))
print(len(pupil_location_three_segmented))

60: Il numero totale di campioni (clip da 140 frame) di dati della pupilla per natural_movie_one.

250: Il numero totale di campioni (clip da 140 frame) di dati della pupilla per natural_movie_three.

**dividiamo i dati in set di addestramento (train), validazione (validation) e test.**
**Dobbiamo addestrare il modello sui trial (ripetizioni) iniziali e testarlo sui trial finali. Questo previene il data leakage, ovvero evita che il modello "veda" dati futuri durante l'addestramento.**

**70% (Train) / 10% (Validation) / 20% (Test)**

In [ ]:
pupil_location_one_train = pupil_location_one_segmented[:-12] #Il set di training iniziale è composto dalle prime 48 clip (i primi 8 trial).
pupil_location_one_validation = pupil_location_one_train[-6:] #Il set di validazione è composto dalle ultime 6 clip del set di training (cioè 1 trial
pupil_location_one_train = pupil_location_one_train[:-6] #Il set di addestramento finale è composto dalle 48 clip iniziali meno le 6 di validazione, lasciando 42 clip (i primi 7 trial).
pupil_location_one_test = pupil_location_one_segmented[-12:]  #Il set di test è composto dalle ultime 12 clip (cioè gli ultimi 2 trial)

pupil_location_three_train = pupil_location_three_segmented[:-50]
pupil_location_three_validation = pupil_location_three_train[-25:] # 1 trial
pupil_location_three_train = pupil_location_three_train[:-25] 
pupil_location_three_test = pupil_location_three_segmented[-50:]

In [ ]:
print(f"# elementi pupil_location_one_train: {len(pupil_location_one_train)}")
print(f"# elementi pupil_location_one_validation: {len(pupil_location_one_validation)}")
print(f"# elementi pupil_location_one_test: {len(pupil_location_one_test)}")

print(f"# elementi pupil_location_three_train: {len(pupil_location_three_train)}")
print(f"# elementi pupil_location_three_validation: {len(pupil_location_three_validation)}")
print(f"# elementi pupil_location_three_test: {len(pupil_location_three_test)}")

In [ ]:
locomotion_speed, _ = ophys_session_data.get_running_speed()
_, pupil_size = ophys_session_data.get_pupil_size()

behavioral_data = np.column_stack((locomotion_speed, pupil_size))

**behavior è un array dove la prima colonna è la velocità di corsa del topo mentre guarda le clip video e la seconda è la dimensione della pupilla per ogni istante dell'esperimento**

In [ ]:
# Estrae il blocco grezzo di "movie one" (trial + pause)
behavior_one = behavioral_data[38751:47810,:]

#Filtra il blocco, tenendo solo i 10 trial (da 840 frame) e scartando le pause (da 66 frame)
behavior_one = extract_stimulus_trials(behavior_one, 840, 66)

print(behavior_one.shape)

* 8400: Frame totali (10 trial × 840 frame/trial).
* 2: Le due colonne di dati (running_speed e pupil_size).

Ora abbiamo un array behavior_one  allineato ai frame di pupil_location_one e ai video cropped_one

In [ ]:
tmp_behavior_three = [behavioral_data[19741:37846,:], behavioral_data[75867:93967,:]]

behavior_three = []
behavior_three.append(extract_stimulus_trials(tmp_behavior_three[0], 3500, 121))
behavior_three.append(extract_stimulus_trials(tmp_behavior_three[1], 3500, 120))

print(behavior_three[0].shape)
print(behavior_three[1].shape)

In [ ]:
behavior_one_segmented = segment_data_into_clips(behavior_one)
behavior_three_segmented = segment_data_into_clips(behavior_three[0])
behavior_three_segmented.extend(segment_data_into_clips(behavior_three[1]))

print(len(behavior_one_segmented))
print(len(behavior_three_segmented))


* 60 clip di dati comportamentali per movie_one.
* 250 clip di dati comportamentali per movie_three.


In [ ]:
behavior_one_train = behavior_one_segmented[:-12]
behavior_one_validation = behavior_one_train[-6:] # 1 trial
behavior_one_train = behavior_one_train[:-6]
behavior_one_test = behavior_one_segmented[-12:]

behavior_three_train = behavior_three_segmented[:-50]
behavior_three_validation = behavior_three_train[-25:] # 1 trial
behavior_three_train = behavior_three_train[:-25] 
behavior_three_test = behavior_three_segmented[-50:]

print(f"# elementi behavior_one_train: {len(behavior_one_train)}")
print(f"# elementi behavior_one_validation: {len(behavior_one_validation)}")
print(f"# elementi behavior_one_test: {len(behavior_one_test)}")

print(f"# elementi behavior_three_train: {len(behavior_three_train)}")
print(f"# elementi behavior_three_validation: {len(behavior_three_validation)}")
print(f"# elementi behavior_three_test: {len(behavior_three_test)}")

## Processamento Dati Neurali (Output/Labels)

**carichiamo i dati di output (labels) che il nostro modello di intelligenza artificiale dovrà imparare a predire.**

**fluorescence_traces = rappresenta la variazione di fluorescenza ($\Delta F$) rispetto alla fluorescenza di base ($F$). In termini semplici, ci dice quanto un neurone è attivo in un dato istante rispetto al suo stato di riposo.**

In [ ]:
# scarica l'attività neurale vera e propria dei 227 neuroni
_, fluorescence_traces = ophys_session_data.get_corrected_fluorescence_traces()
print(fluorescence_traces.shape)

* 227: È il numero di neuroni. Ogni riga di questo array è la traccia di attività di un singolo neurone
* 115735: numero totale di frame dell'intero esperimento (non solo video).
    * (Corrisponde alla lunghezza dei dati della pupilla e di corsa, visti nelle Celle precedenti).

**estraiamo l'attività dei 227 neuroni esattamente durante la proiezione del natural_movie_one, scartando le pause.**

In [ ]:
neural_activity_movie_one = fluorescence_traces[:, 38751:47810]
neural_activity_movie_one = extract_stimulus_trials(neural_activity_movie_one, 840, 66, "fluorescence_traces")
print(neural_activity_movie_one.shape)

* 227: I 227 neuroni.

* 8400: Il numero totale di frame di attività estratti

    * 10 ripetizioni (trial) × 840 frame/trial = 8400 frame.

In [ ]:
#Movie Three è stato mostrato in due momenti diversi della sessione
tmp_neural_activity_three = [fluorescence_traces[:, 19741:37846,], fluorescence_traces[:, 75867:93967]]

neural_activity_movie_three = []
neural_activity_movie_three.append(extract_stimulus_trials(tmp_neural_activity_three[0], 3500, 121, "fluorescence_traces"))
neural_activity_movie_three.append(extract_stimulus_trials(tmp_neural_activity_three[1], 3500, 120, "fluorescence_traces"))

print(neural_activity_movie_three[0].shape)
print(neural_activity_movie_three[1].shape)

17.500 frame totali / 3.500 frame a video = 5 Trial per blocco
Tot 10 Trial totali per Movie Three

In [ ]:
# Segmenta l'attività 'fluorescence_traces' di movie_one (8400 frame) in clip da 140
neural_activity_one_segmented = segment_data_into_clips(neural_activity_movie_one,1)

# Segmenta il primo blocco di 'fluorescence_traces' di movie_three (17500 frame)
neural_activity_three_segmented = segment_data_into_clips(neural_activity_movie_three[0],1)

# Aggiunge i segmenti del secondo blocco di 'fluorescence_traces' di movie_three (altri 17500 frame)
neural_activity_three_segmented.extend(segment_data_into_clips(neural_activity_movie_three[1],1))

print(len(neural_activity_one_segmented))
print(len(neural_activity_three_segmented))

Il modello non elabora tutto il video intero in una volta, ma in clip.

Movie One: 8400 frame totali / 140 frame per clip = 60 Clip.
* Poiché ci sono 10 Trial, ogni Trial è composto da 6 Clip ($60/10=6$).

Movie Three: (17500 + 17500) frame / 140 frame per clip = 250 Clip.
* Poiché ci sono 10 Trial, ogni Trial è composto da 25 Clip ($250/10=25$).

**In totale abbiamo 310 campioni (60 + 250) pronti per essere suddivisi in set di addestramento, validazione e test**

In [ ]:
#SUDDIVISIONE Train / Validation / Test

neural_activity_one_train = neural_activity_one_segmented[:-12]
neural_activity_one_validation = neural_activity_one_train[-6:] # 1 trial
neural_activity_one_train = neural_activity_one_train[:-6]
neural_activity_one_test = neural_activity_one_segmented[-12:]

neural_activity_three_train = neural_activity_three_segmented[:-50]
neural_activity_three_validation = neural_activity_three_train[-25:] # 1 trial
neural_activity_three_train = neural_activity_three_train[:-25]
neural_activity_three_test = neural_activity_three_segmented[-50:]

print(f"# elementi neural_activity_one_train: {len(neural_activity_one_train)}")
print(f"# elementi neural_activity_one_validation: {len(neural_activity_one_validation)}")
print(f"# elementi neural_activity_one_test: {len(neural_activity_one_test)}")

print(f"# elementi neural_activity_three_train: {len(neural_activity_three_train)}")
print(f"# elementi neural_activity_three_validation: {len(neural_activity_three_validation)}")
print(f"# elementi neural_activity_three_test: {len(neural_activity_three_test)}")

**Analisi Movie One:**
* Test: Ultime 12 clip. (12 clip / 6 clip per trial = 2 Trial).
* Validation: Ultime 6 clip dei rimanenti. (6 / 6 = 1 Trial).
* Train: Rimanenti 42 clip. (42 / 6 = 7 Trial).

Totale: 7+1+2 = 10 Trial.

**Analisi Movie Three:**
* Test: Ultime 50 clip. (50 clip / 25 clip per trial = 2 Trial).
* Validation: Ultime 25 clip dei rimanenti. (25 / 25 = 1 Trial).
* Train: Rimanenti 175 clip. (175 / 25 = 7 Trial).

# Preparazione Finale del Dataset

## Pulizia Dati

rimuoviamo i valori corrotti (`NaN`) dal dataset per evitare crash durante l'addestramento della rete neurale.

In [ ]:
def impute_missing_data(items,mean = None):
    """
    Funzione per la pulizia dei dati (NaN e 0) in due modalità.
    Modalità 1 (se 'mean' è None): Sostituisce i valori NaN (Not a Number) con 0.
    Modalità 2 (se 'mean' è fornito): Tratta 0 come valore mancante (es. tracking perso) 
    e lo sostituisce con la media fornita (imputazione).
    """
    items = np.array(items)  # Convert to a NumPy array (if it is not already)
    if mean:
         items[items == 0] = mean  
    else:
        items[np.isnan(items)] = 0  
    return items

**Sostituiamo i valori NaN con 0**

In [ ]:
# TRAIN
behavior_one_train = impute_missing_data(behavior_one_train)
behavior_three_train = impute_missing_data(behavior_three_train)

pupil_location_one_train = impute_missing_data(pupil_location_one_train)
pupil_location_three_train = impute_missing_data(pupil_location_three_train)

neural_activity_one_train = impute_missing_data(neural_activity_one_train)
neural_activity_three_train = impute_missing_data(neural_activity_three_train)

# VALIDATION
behavior_one_validation = impute_missing_data(behavior_one_validation)
behavior_three_validation = impute_missing_data(behavior_three_validation)

pupil_location_one_validation = impute_missing_data(pupil_location_one_validation)
pupil_location_three_validation = impute_missing_data(pupil_location_three_validation)

neural_activity_one_validation = impute_missing_data(neural_activity_one_validation)
neural_activity_three_validation = impute_missing_data(neural_activity_three_validation)

# TEST
behavior_one_test = impute_missing_data(behavior_one_test)
behavior_three_test = impute_missing_data(behavior_three_test)

pupil_location_one_test = impute_missing_data(pupil_location_one_test)
pupil_location_three_test = impute_missing_data(pupil_location_three_test)

neural_activity_one_test = impute_missing_data(neural_activity_one_test)
neural_activity_three_test = impute_missing_data(neural_activity_three_test)

## Imputazione e Normalizzazione Finale

Calcolo parametri di riferimento per la normalizzazione dei dati

**esclusivamente sul Training Set** per evitare il *Data Leakage*. Gli stessi valori verranno poi applicati per scalare Validation e Test set.

1.  **Behavior & Pupil:** Estraiamo `Min` e `Max`. Verranno usati per una **Min-Max Normalization** (range 0-1).
2.  **Attività Neurale:** Estraiamo `Mean` e `Std`. Verranno usati per la **Z-Score**, più robusta per segnali neurali che possono presentare picchi elevati

In [ ]:
def calculate_statistics(items):
    """
    Concatena tutti i campioni in un unico array 1D prima di calcolare 
    le statistiche globali.
    Calcola le statistiche aggregate (media, deviazione standard, min, max) 
    per un'intera collezione di campioni (es. una lista di clip video).

    Args:
        items (list): Una lista di array NumPy (es. clip).
    """
    all_values = np.concatenate([item.astype(np.float32).ravel() for item in items])

    mean = np.mean(all_values)
    std = np.std(all_values)
    min_val = np.min(all_values)
    max_val = np.max(all_values)

    return mean, std, min_val, max_val

In [ ]:
# Calcola le statistiche SOLO SUL TRAIN SET
mean_b1_train, _, min_behavior1, max_behavior1 = calculate_statistics(behavior_one_train)
mean_b3_train, _, min_behavior3, max_behavior3 = calculate_statistics(behavior_three_train)

mean_p1_train, _, min_pupil1, max_pupil1 = calculate_statistics(pupil_location_one_train)
mean_p3_train, _, min_pupil3, max_pupil3 = calculate_statistics(pupil_location_three_train)

mean_neural_activity1_train, neural_activity_std1, _, _ = calculate_statistics(neural_activity_one_train)
mean_neural_activity3_train, neural_activity_std3, _, _ = calculate_statistics(neural_activity_three_train)

trattiamo tutti i valori 0 (sia quelli originali che quelli ex-NaN) come dati mancanti. 

Per i dati comportamentali (es. pupil_location, behavior), uno 0 spesso indica un fallimento del tracciamento (es. l'occhio del topo si è chiuso) e non un valore reale.

Usiamo la funzione fill_missing_values (in Modalità 2) per sostituire tutti questi zeri con la media statistica calcolata **esclusivamente sul set di addestramento** (mean_b1_train) per prevenire il data leakage.

* il modello viene addestrato e validato senza mai "sbirciare" informazioni statistiche provenienti dai dati futuri

In [ ]:
# --- IMPUTAZIONE CON STATISTICHE DEL TRAIN SET ---

# TRAIN
behavior_one_train = impute_missing_data(behavior_one_train, mean_b1_train)
behavior_three_train = impute_missing_data(behavior_three_train, mean_b3_train)

pupil_location_one_train = impute_missing_data(pupil_location_one_train, mean_p1_train)
pupil_location_three_train = impute_missing_data(pupil_location_three_train, mean_p3_train)

neural_activity_one_train = impute_missing_data(neural_activity_one_train, mean_neural_activity1_train)
neural_activity_three_train = impute_missing_data(neural_activity_three_train, mean_neural_activity3_train)

# VALIDATION
behavior_one_validation = impute_missing_data(behavior_one_validation, mean_b1_train)
behavior_three_validation = impute_missing_data(behavior_three_validation, mean_b3_train)

pupil_location_one_validation = impute_missing_data(pupil_location_one_validation, mean_p1_train)
pupil_location_three_validation = impute_missing_data(pupil_location_three_validation, mean_p3_train)

neural_activity_one_validation = impute_missing_data(neural_activity_one_validation, mean_neural_activity1_train)

neural_activity_three_validation = impute_missing_data(neural_activity_three_validation, mean_neural_activity3_train)

# TEST
behavior_one_test = impute_missing_data(behavior_one_test, mean_b1_train)
behavior_three_test = impute_missing_data(behavior_three_test, mean_b3_train)

pupil_location_one_test = impute_missing_data(pupil_location_one_test, mean_p1_train)
pupil_location_three_test = impute_missing_data(pupil_location_three_test, mean_p3_train)

neural_activity_one_test = impute_missing_data(neural_activity_one_test, mean_neural_activity1_train)

neural_activity_three_test = impute_missing_data(neural_activity_three_test, mean_neural_activity3_train)

In [ ]:
def standardize_neural_activity(items, std):
    """
    Normalizza una lista di item dividendoli per la deviazione standard (std) globale.
    Questo metodo è usato specificamente per normalizzare le tracce fluorescence_traces (attività neurale).
    """
    return [item / std for item in items]


**Applicazione della Normalizzazione**

In questo passaggio trasformiamo i dati grezzi in input pronti per la rete neurale, applicando:

1.  **Min-Max Scaling (per Comportamento e Pupilla):**
    * Ridimensiona i dati nell'intervallo `[0, 1]`.

2.  **Standardizzazione / Scaling (per Attività Neurale):**
    * Divide i dati per la deviazione standard (`neural_activity / std`).
    * Evita che un singolo picco elevato "schiacci" tutto il resto del segnale a zero, come accadrebbe con il Min-Max.

I set di **Validation** vengono normalizzati utilizzando le statistiche (`min`, `max`, `std`) calcolate sul **Training Set**. Questo garantisce che il modello veda i nuovi dati attraverso la stessa "scala di valori" appresa durante l'addestramento.

In [ ]:
# --- NORMALIZZAZIONE CON STATISTICHE DEL TRAIN SET ---

# Normalization (Train)
normalized_behavior_one = apply_min_max_normalization(behavior_one_train, min_behavior1, max_behavior1)
normalized_behavior_three = apply_min_max_normalization(behavior_three_train, min_behavior3, max_behavior3)
normalized_pupil_location_one = apply_min_max_normalization(pupil_location_one_train, min_pupil1, max_pupil1)
normalized_pupil_location_three = apply_min_max_normalization(pupil_location_three_train, min_pupil3, max_pupil3)

normalized_neural_activity_one = standardize_neural_activity(neural_activity_one_train, neural_activity_std1)
normalized_neural_activity_three = standardize_neural_activity(neural_activity_three_train, neural_activity_std3)


# Normalization (Validation)
# Applichiamo le stesse statistiche (min_behavior1, neural_activity_std1, etc.) usate per il train set
normalized_behavior_one_val = apply_min_max_normalization(behavior_one_validation, min_behavior1, max_behavior1)
normalized_behavior_three_val = apply_min_max_normalization(behavior_three_validation, min_behavior3, max_behavior3)
normalized_pupil_location_one_val = apply_min_max_normalization(pupil_location_one_validation, min_pupil1, max_pupil1)
normalized_pupil_location_three_val = apply_min_max_normalization(pupil_location_three_validation, min_pupil3, max_pupil3)

normalized_neural_activity_one_val = standardize_neural_activity(neural_activity_one_validation, neural_activity_std1)
normalized_neural_activity_three_val = standardize_neural_activity(neural_activity_three_validation, neural_activity_std3)

## Salvataggio dei Dati Elaborati

In [ ]:
def save_dataset_shards(items, start_idx, output_dir, label="", trials=1):
    index = start_idx
    total = trials * len(items)
    with tqdm(total=total, desc=f"Saving {label}", unit=label) as pbar:
        for i in range(trials):
            for item in items:
                filename = f"{index}.npy"
                filepath = os.path.join(output_dir, filename)
                np.save(filepath, item)
                index += 1
                pbar.update(1)
    return index

In [ ]:
############################### VIDEOS ################################
output_dir = '/kaggle/working/test/data/videos'
os.makedirs(output_dir, exist_ok=True)

trials = 2

# Saving 
next_index = save_dataset_shards(items=normalized_one, start_idx= 0, trials=trials, label="normalized_one", output_dir=output_dir)
_ = save_dataset_shards(items=normalized_three, start_idx=next_index, trials=trials, label="normalized_three", output_dir=output_dir)

############################### PUPIL LOCATION ################################
output_dir = '/kaggle/working/test/data/pupil_center'
os.makedirs(output_dir, exist_ok=True)

# Normalization of pupil_tracking_data
normalized_pupil_location_one = apply_min_max_normalization(pupil_location_one_test, min_pupil1, max_pupil1)
normalized_pupil_location_three = apply_min_max_normalization(pupil_location_three_test, min_pupil3, max_pupil3)

# Saving
next_index_pupil_center = save_dataset_shards(items=normalized_pupil_location_one, start_idx=0, output_dir=output_dir,label="normalized_pupil_location_one")
_ = save_dataset_shards(items=normalized_pupil_location_three, start_idx=next_index_pupil_center, output_dir=output_dir,label="normalized_pupil_location_three")

############################### BEHAVIOR ################################
output_dir = '/kaggle/working/test/data/behavioral_data'
os.makedirs(output_dir, exist_ok=True)

# Normalization of behavioral_data
normalized_behavior_one = apply_min_max_normalization(behavior_one_test, min_behavior1, max_behavior1)
normalized_behavior_three = apply_min_max_normalization(behavior_three_test, min_behavior3, max_behavior3)

# Saving
next_index_behavior = save_dataset_shards(items=normalized_behavior_one, start_idx=0,output_dir=output_dir,label="normalized_behavior_one")
_ = save_dataset_shards(items=normalized_behavior_three, start_idx=next_index_behavior, output_dir=output_dir,label="normalized_behavior_three")

############################### LABELS ################################
output_dir = '/kaggle/working/test/data/labels'
os.makedirs(output_dir, exist_ok=True)


# Normalization of fluorescence_traces
normalized_neural_activity_one = standardize_neural_activity(neural_activity_one_test, neural_activity_std1)
normalized_neural_activity_three = standardize_neural_activity(neural_activity_three_test, neural_activity_std3)

# Saving
next_index_neural_activity = save_dataset_shards(items=normalized_neural_activity_one, start_idx=0, output_dir=output_dir,label="normalized_neural_activity_one")
_ = save_dataset_shards(items=normalized_neural_activity_three, start_idx=next_index_neural_activity ,output_dir=output_dir,label="normalized_neural_activity_three")

In [ ]:
output_dir_videos = '/kaggle/working/train/data/videos'
output_dir_behavior = '/kaggle/working/train/data/behavioral_data'
output_dir_pupil_center = '/kaggle/working/train/data/pupil_center'
output_dir_neural_activity = '/kaggle/working/train/data/labels'

output_dir_videos_val = '/kaggle/working/validation/data/videos'
output_dir_behavior_val = '/kaggle/working/validation/data/behavioral_data'
output_dir_pupil_center_val = '/kaggle/working/validation/data/pupil_center'
output_dir_neural_activity_val = '/kaggle/working/validation/data/labels'

os.makedirs(output_dir_videos, exist_ok=True)
os.makedirs(output_dir_behavior, exist_ok=True)
os.makedirs(output_dir_pupil_center, exist_ok=True)
os.makedirs(output_dir_neural_activity, exist_ok=True)

os.makedirs(output_dir_videos_val, exist_ok=True)
os.makedirs(output_dir_behavior_val, exist_ok=True)
os.makedirs(output_dir_pupil_center_val, exist_ok=True)
os.makedirs(output_dir_neural_activity_val, exist_ok=True)

# Saving TRAINING
next_index = save_dataset_shards(items=normalized_one, start_idx=0, trials=7, label="normalized_one", output_dir=output_dir_videos)
_ = save_dataset_shards(items=normalized_three, start_idx=next_index, trials=7, label="normalized_three", output_dir=output_dir_videos)

next_index_behavior = save_dataset_shards(items=normalized_behavior_one, start_idx=0, output_dir=output_dir_behavior, label="normalized_behavior_one")
_ = save_dataset_shards(items=normalized_behavior_three, start_idx=next_index_behavior, output_dir=output_dir_behavior, label="normalized_behavior_three")

next_index_pupil_center = save_dataset_shards(items=normalized_pupil_location_one, start_idx=0, output_dir=output_dir_pupil_center, label="normalized_pupil_location_one")
_ = save_dataset_shards(items=normalized_pupil_location_three, start_idx=next_index_pupil_center, output_dir=output_dir_pupil_center, label="normalized_pupil_location_three")

next_index_neural_activity = save_dataset_shards(items=normalized_neural_activity_one, start_idx=0, output_dir=output_dir_neural_activity, label="normalized_neural_activity_one")
_ = save_dataset_shards(items=normalized_neural_activity_three, start_idx=next_index_neural_activity, output_dir=output_dir_neural_activity, label="normalized_neural_activity_three")

# Saving VALIDATION
next_index_val = save_dataset_shards(items=normalized_one, start_idx=0, trials=1, label="normalized_one_val", output_dir=output_dir_videos_val)
_ = save_dataset_shards(items=normalized_three, start_idx=next_index_val, trials=1, label="normalized_three_val", output_dir=output_dir_videos_val)

next_index_behavior_val = save_dataset_shards(items=normalized_behavior_one_val, start_idx=0, output_dir=output_dir_behavior_val, label="normalized_behavior_one_val")
_ = save_dataset_shards(items=normalized_behavior_three_val, start_idx=next_index_behavior_val, output_dir=output_dir_behavior_val, label="normalized_behavior_three_val")

next_index_pupil_center_val = save_dataset_shards(items=normalized_pupil_location_one_val, start_idx=0, output_dir=output_dir_pupil_center_val, label="normalized_pupil_location_one_val")
_ = save_dataset_shards(items=normalized_pupil_location_three_val, start_idx=next_index_pupil_center_val, output_dir=output_dir_pupil_center_val, label="normalized_pupil_location_three_val")

next_index_neural_activity_val = save_dataset_shards(items=normalized_neural_activity_one_val, start_idx=0, output_dir=output_dir_neural_activity_val, label="normalized_neural_activity_one_val")
_ = save_dataset_shards(items=normalized_neural_activity_three_val, start_idx=next_index_neural_activity_val, output_dir=output_dir_neural_activity_val, label="normalized_neural_activity_three_val")

## Creazione classe MouseDataset

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset

class MouseDataset(Dataset):
    def __init__(self, root_dir):
        # percorsi per le 4 cartelle di dati
        self.videos_dir = os.path.join(root_dir, 'videos')
        self.pupil_dir = os.path.join(root_dir, 'pupil_center')
        self.behavior_dir = os.path.join(root_dir, 'behavioral_data')
        self.labels_dir = os.path.join(root_dir, 'labels')
        
        # Creo una lista "master" di tutti i file ID (es. '0', '1', '2', ...)
        #    ordinandoli numericamente (es. 1, 2, ... 10, 11)
        self.file_ids = sorted(
            [f.replace('.npy', '') for f in os.listdir(self.videos_dir) if f.endswith('.npy')],
            key=lambda x: int(x)
        )

    def __len__(self):
        #ritorna quanti campioni totali ci sono
        return len(self.file_ids)

    def __getitem__(self, i):
        #Carica un campione specifico i
        file_id = self.file_ids[i]

        # Carica i 4 file .npy corrispondenti usando lo STESSO file_id.
        # Questo garantisce che video, pupilla, comportamento e label
        # siano perfettamente allineati temporalmente.
        video = np.load(os.path.join(self.videos_dir, f'{file_id}.npy'))
        pupil = np.load(os.path.join(self.pupil_dir, f'{file_id}.npy'))
        behavioral_data = np.load(os.path.join(self.behavior_dir, f'{file_id}.npy'))
        label = np.load(os.path.join(self.labels_dir, f'{file_id}.npy'))

        #Aggiunge una dimensione "canale" al video. 
        # Forma: (140, 36, 64) -> (1, 140, 36, 64) [Canali, Tempo, Altezza, Larghezza]
        video = video[np.newaxis, ...]

        #Restituisce un dizionario di Tensori PyTorch
        return {
            'video': torch.from_numpy(video).float(),
            'pupil_center': torch.from_numpy(pupil).float().transpose(1,0),
            'behavioral_data': torch.from_numpy(behavioral_data).float().transpose(1,0),
            'labels': torch.from_numpy(label).float()
        }

**creiamo le tre istanze dei set di dati (addestramento, validazione e test) utilizzando la classe MouseDataset.
Questi oggetti sono ora pronti per essere passati a un DataLoader di PyTorch**

In [ ]:
train_set = MouseDataset('/kaggle/working/train/data')
validation_set = MouseDataset('/kaggle/working/validation/data')
test_set = MouseDataset('/kaggle/working/test/data')

# Configurazione del Modello (ViV1T)

## Calcolo delle Coordinate Spaziali

In [ ]:
# Recupera gli ID associati ai 227 neuroni del topo
cell_specimen_ids = ophys_session_data.get_cell_specimen_ids() 

# Per ogni neurone recupera la rispettiva maschera
roi_masks = ophys_session_data.get_roi_mask_array(cell_specimen_ids=cell_specimen_ids) 
print(f"Forma delle maschere ROI: {roi_masks.shape}")

neuron_centroids = []
for i, cell_id in enumerate(cell_specimen_ids):
    mask = roi_masks[i]

    # Trova tutti i pixel che compongono la sagoma del neurone
    y_coords, x_coords = np.where(mask)

    if len(x_coords) > 0 and len(y_coords) > 0:
        # Calcola il centro geometrico (centroide) della sagoma
        centroid_x = np.mean(x_coords)
        centroid_y = np.mean(y_coords)
        neuron_centroids.append([centroid_x, centroid_y])
    else:
        # Se la maschera è vuota, aggiunge coordinate nulle
        neuron_centroids.append([0.0, 0.0])

# Converte la lista di coordinate in un tensore PyTorch
neuron_centroids = np.array(neuron_centroids, dtype=np.float32)
neurons_coordinates_tensor = torch.from_numpy(neuron_centroids)

print(f"Coordinate dei neuroni:")
print(f"Forma: {neuron_centroids.shape}")
print(f"Prime 5 coordinate:\n {neuron_centroids[:5]}")

Processiamo le maschere binarie (ROI) per estrarre le coordinate (x, y) del centroide di ciascun neurone e li inseriamo all'interno di un tensore che verrà utilizzato dal componente del modello Readout che utilizzerà queste coordinate per mappare le feature visive estratte dal Core alla specifica posizione fisica del neurone.

Questo evita che la rete debba apprendere la posizione dei neuroni da zero, accelerando significativamente la convergenza dell'addestramento

**Il Tensore finale ha 227 righe (una per ogni neurone) e 2 colonne (una per la coordinata x e una per la y).**

In [ ]:
# Recupera gli ID dei primi tre neuroni
cids = ophys_session_data.get_cell_specimen_ids()[:3]
selected_roi_masks = ophys_session_data.get_roi_mask_array(cell_specimen_ids=cids)

# Mostra ogni singola maschera
f, axes = plt.subplots(1, len(cids)+2, figsize=(15, 3))
for ax, roi_mask, cid in zip(axes[:-2], selected_roi_masks, cids):
    ax.imshow(roi_mask, cmap='gray') 
    ax.set_title('cell %d' % cid)

# Crea una maschera cumulativa di tutte le ROI nell'esperimento
all_roi_masks = ophys_session_data.get_roi_mask_array()
combined_mask = all_roi_masks.max(axis=0)
axes[-2].imshow(combined_mask, cmap='gray')
axes[-2].set_title('all ROIs')

# show the movie max projection
max_projection = ophys_session_data.get_max_projection()
axes[-1].imshow(max_projection, cmap='gray')
axes[-1].set_title('max projection')
plt.show()

* grafico 1-3: Questi primi tre grafici mostrano la forma e la posizione esatta di quei 3 neuroni specifici all'interno del campo visivo 512x512
* all ROIs: "appiattisce" i 227 livelli in un'unica immagine, mostrando la mappa completa di ogni singolo neurone registrato in questo esperimento
* max projection: Mostra il valore più luminoso che ogni pixel abbia mai raggiunto. È utile per vedere la struttura del tessuto e i vasi sanguigni

è necessario trasformare le coordinate spaziali dei neuroni dell'Allen Brain Observatory per renderle compatibili con il modello di riferimento SENSORIUM

In [ ]:
def convert_to_sensorium(roi_coords_tensor, factor=1.24):
    """
    Converte le coordinate dal sistema in alto a sinistra al sistema Sensorium (alto a destra, con x negativa).
    """

    x_new = roi_coords_tensor[:, 0] * factor   
    y_new = roi_coords_tensor[:, 1] * factor   
    
    x_sensorium = -620 + x_new
    y_sensorium = -y_new
    
    return torch.stack([x_sensorium, y_sensorium], dim=1)

neurons_coordinates_tensor = convert_to_sensorium(neurons_coordinates_tensor)

In [ ]:
print(f"Prime 5 coordinate:\n {neurons_coordinates_tensor[:5]}")

## Installazione e Caricamento del Modello Base

In [ ]:
%%capture

!git clone https://github.com/bryanlimy/ViV1T.git
%cd ViV1T
!pip install -e .

In [ ]:
import importlib.util
import numpy as np
import argparse
import sys

sys.path.insert(0, '/kaggle/input/viv1t/transformers/default/1')
args_path = '/kaggle/input/viv1t/transformers/default/1/args.py'
spec = importlib.util.spec_from_file_location("args", args_path)
args = importlib.util.module_from_spec(spec)
spec.loader.exec_module(args)

args_dict = args.args_dict

sys.path.append('./src')

In [ ]:
%%capture
!pip install --quiet --force-reinstall "scipy==1.11.4"

**I pesi shifters e readouts del modello Sensorium sono inutili per noi, perché erano addestrati per un altro topo e altri neuroni.**

**carichiamo nel modello:**
* Argomenti (args): Definiscono la struttura del Core (il Transformer), garantendo che sia compatibile con i pesi pre-addestrati che caricheremo
* Coordinate (neuron_coordinates): Definiscono la struttura del Readout

In [ ]:
from viv1t.model import Model

args = argparse.Namespace(**args_dict) #per usare la dot-notation

neuron_coordinates = {
    'A': neurons_coordinates_tensor
}

viv1t = Model(args, neuron_coordinates=neuron_coordinates)

In [ ]:
checkpoint = torch.load("/kaggle/input/viv1t/transformers/default/1/model_state.pt", map_location=args.device, weights_only=False) # modello Sensorium
state_dict = checkpoint['model']

# filtriamo i pesi caricando solo i pesi del "core" (la parte di elaborazione video)
filtered_checkpoint = {}
for key, value in state_dict.items():
    if key.startswith('core.'):
        filtered_checkpoint[key] = value

viv1t.load_state_dict(filtered_checkpoint, strict=False) # strict=False ==> Carica i pesi del Core e ignora il fatto che mancano i pesi per il nuovo Readout

# Sposta l'intero modello sulla GPU
viv1t = viv1t.to(args.device)

**Architettura del modello viv1t**

In [ ]:
viv1t

Possiamo dividerlo in 3 parti principali, che lavorano in sequenza.

**core [Video Vision Transformer (ViViT)]**: Si occupa di guardare e capire le clip video.

è composto da

* tokenizer: Divide il video in Tubelet (patch tridimensionali spazio-tempo) e le converte in token  che il Transformer può capire.

* spatial_transformer: analizza i token all'interno di un **singolo** frame per capire le relazioni spaziali ("cosa c'è nell'immagine?").

* temporal_transformer: analizza i token **tra** i frame per capire il movimento e le relazioni temporali ("cosa sta succedendo nel tempo?").

* rearrange: Riordina i dati di output in una feature map pronta per essere letta.

**MLPShifters**: riceve i dati comportamentali (posizione della pupilla e velocità di corsa) e li usa per modificare spostare l'immagine per compensare il fatto che il topo stava guardando leggermente a destra o a sinistra

**readouts (Gaussian2DReadout)**: Invece di collegare tutti i pixel a tutti i neuroni (che richiederebbe troppi parametri), questo approccio assume che ogni neurone guardi solo una piccola porzione dello schermo. Per pesare l'attenzione in una zona specifica dell'immagine il modello utilizza una funzione Gaussiana 

**output_activation: Exponential()**
Applica una funzione esponenziale all'output per assicurarsi che tutte le previsioni dell'attività neurale siano positive, proprio come i dati reali "neural_activity" (la fluorescenza non può essere negativa).

In [ ]:
torch.cuda.empty_cache()
torch.cuda.synchronize()
free, _ = torch.cuda.mem_get_info()
print(f"Memoria GPU libera: {free / 1e9:.3f} GB")

# Impostazione del Finetuning (PEFT e Trainer)

la libreria PEFT (usata per il LoRA) è nata per i modelli di linguaggio (come GPT) e si aspetta che il metodo forward del modello accetti parametri standard come input_ids.

Il nostro modello ViV1T, invece, ha un metodo forward personalizzato che richiede parametri specifici come inputs (il video), mouse_id, behaviors, ecc.

Quindi non è possibile utilizzare direttamente get_peft_model a causa di un problema di compatibilità

La soluzione è creare un Wrapper che adatta l'interfaccia di ViV1T a quella attesa da PEFT.

In [ ]:
"""
%%capture
!pip install --quiet --upgrade "transformers" "peft" "accelerate"
!pip install --quiet "protobuf==3.20.3"
"""

## Wrapper per PEFT (LoRA)

In [ ]:
import torch
import torch.nn as nn 
from peft import get_peft_model, LoraConfig, TaskType
import os
import warnings


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #mostra solo messaggi di errore
warnings.filterwarnings('ignore')

class ViV1TWrapper(nn.Module):
    """Wrapper per adattare ViV1T all'interfaccia PEFT"""
    
    def __init__(self, vivit_model):
        super().__init__()
        self.vivit_model = vivit_model
        
    def forward(self, input_ids=None, inputs=None, mouse_id=None, behaviors=None, pupil_centers=None, **kwargs):
        """
        gestisce sia l'interfaccia PEFT che quella originale
        """
        if inputs is not None:
            return self.vivit_model(
                inputs=inputs,
                mouse_id=mouse_id,
                behaviors=behaviors,
                pupil_centers=pupil_centers
            )
        else:
            raise ValueError("Missing required parameters for ViV1T model")

## Configurazione del Trainer e Logging

In [ ]:
import os
os.chdir('/kaggle/working')

print("Current directory:", os.getcwd())

In [ ]:
%%capture

!pip install huggingface_hub transformers
!pip install wandb -qqq

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient
import os
from huggingface_hub import HfApi

user_secrets = UserSecretsClient() 

wandb_key = user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_API_KEY"] = wandb_key

if wandb_key:
    wandb.login(key=wandb_key)
    print("Accesso a WandB effettuato con successo")


hf_hub_token = user_secrets.get_secret("HF_HUB_TOKEN")
os.environ["HF_HUB_TOKEN"] = hf_hub_token

if hf_hub_token:
    print("token hugging face hub recuperato con successo")

## Wrapper per Trainer di Hugging Face

Mentre il ViV1TWrapper rende ViV1T compatibile con la libreria PEFT, questo ViViTTrainerWrapper rende il modello (già avvolto da PEFT) compatibile con il Trainer di Hugging Face.

Ha due compiti principali:

* Fare da "ponte" tra il MouseDataset e il modello.
* Calcolare la loss del modello.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset
from types import SimpleNamespace
import os
import warnings

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

class ViViTTrainerWrapper(nn.Module):
    def __init__(self, lora_model, mouse_id='A'):
        super().__init__()
        self.lora_model = lora_model
        self.mouse_id = mouse_id
        self.mse_loss = nn.MSELoss()
        
    def forward(self, video, behavioral_data, pupil_center, labels=None, **kwargs): 
        """
        ora accetta gli output del MouseDataset.
        'labels' è opzionale (None) perché non è presente durante l'inferenza,
        ma è presente durante il training e la valutazione.
        """
        
        # Non c'è più bisogno di kwargs.get()
        # 'inputs' nel modello interno (lora_model) si aspetta il tensore video
        predictions, _ = self.lora_model(
            inputs=video,
            mouse_id=self.mouse_id,
            behaviors=behavioral_data,
            pupil_centers=pupil_center
        )
        
        loss = None
        if labels is not None:
            # Allineamento temporale
            # Il modello riceve 140 frame, ma ne predice solo 66.
            min_frames = 66 # clip neurali valide, le altre hanno schermo grigio
            # Estraggo gli ultimi 66 frame dalle etichette (labels)
            labels_aligned = labels[..., -min_frames:]
            # Estraggo gli ultimi 66 frame dalle predizioni
            predictions_aligned = predictions[..., -min_frames:]

            #Calcolo l'errore (MSE) solo su quei 66 frame allineati
            loss = self.mse_loss(predictions_aligned, labels_aligned) 
        
        return {
            "loss": loss, #valore scalare dell'errore (da minimizzare).
            "logits": predictions,   #Le predizioni grezze del modello (per calcolare le metriche)
        }

**La classe CustomViViTTrainer estende il Trainer standard della libreria Hugging Face Transformers**

**Lo scopo è sovrascrivere il metodo prediction_step.**

## Custom Trainer per la Valutazione (Override)

Questa classe estende il `Trainer` standard di Hugging Face per gestire correttamente l'output personalizzato del nostro modello durante la fase di validazione e test.

Poiché il nostro `ViViTTrainerWrapper` restituisce un dizionario custom (`{'loss': ..., 'logits': ...}`) invece delle tuple standard attese da Hugging Face, è necessario sovrascrivere il metodo **`prediction_step`**.

Ritorna la tupla `(loss, logits, labels)` in modo che successivamente la funzione `compute_metrics` riceva i dati corretti.

In [ ]:
class CustomViViTTrainer(Trainer):
   
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    
    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        """
        Viene chiamato automaticamente durante:
           - trainer.evaluate() per calcolare metriche sul validation set
        """  
        # Prepara gli input (es. li sposta sulla GPU)
        inputs = self._prepare_inputs(inputs) 

        # Disattiva il calcolo dei gradienti per risparmiare memoria
        with torch.no_grad():
            with self.compute_loss_context_manager(): 
                # Esegue il forward pass chiamando il nostro ViViTTrainerWrapper
                outputs = model(**inputs)
                # Recupera la loss e i logits (predizioni) calcolati dal wrapper
                loss = outputs["loss"]
                logits = outputs["logits"]
        
        if prediction_loss_only: # Se il trainer è configurato per calcolare solo la loss
            return (loss, None, None)

        # Recupera le labels reali dal batch di input
        labels = inputs.get('labels')
        
        return (loss, logits, labels) 

## Definizione delle Metriche di Valutazione

Sarà chiamata dal CustomViViTTrainer alla fine di ogni ogni step di valutazione (epoca) per misurare la performance del modello sul set di validazione.

## Metrica di Valutazione: Correlazione di Pearson

In ambito neuroscientifico, l'errore quadratico (MSE) non è sufficiente perchè a noi ci interessa sapere se il modello predice correttamente la **forma d'onda** (il pattern temporale) di quando un neurone si attiva.

La correlazione di Pearson misura la similarità della forma d'onda.

* 1.0: Le due linee salgono e scendono in perfetta sincronia (predizione perfetta).

* 0.0: Non c'è relazione (il modello tira a indovinare).

* -1.0: Opposto (quando il neurone si attiva, il modello predice che si spegne).


*La funzione restituisce:*
* **`eval_average_single_trial_correlation` (KPI Principale):** La media delle correlazioni di Pearson su tutti i trial. È il valore di riferimento per stabilire la qualità del modello (più alto è meglio).
* **`eval_single_trial_std` (Stabilità):** La deviazione standard delle correlazioni. Indica quanto le performance oscillano tra un video e l'altro (più basso è meglio, indica coerenza).
* **`eval_num_examples` (Debug):** Il conteggio dei trial validi effettivamente utilizzati nel calcolo. Utile per verificare che non siano stati scartati troppi campioni a causa di errori numerici (es. `NaN`).

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred 
    
    # Allineamento Temporale
    min_frames = 66 # come nel wrapper della loss valutiamo solo gli ultimi 66 frame, che sono quelli validi.
    labels_aligned = labels[..., -min_frames:]
    predictions_aligned = predictions[..., -min_frames:]
    
    
    example_correlations = [] 
    
    # Itero su ogni clip nel Batch
    for example_idx in range(labels_aligned.shape[0]):

        # Trasforniamo i dati da (neuroni, tempo) a (tempo, neuroni)
        # per facilitare il loop successivo.
        y_true_example = labels_aligned[example_idx].T  
        y_pred_example = predictions_aligned[example_idx].T
        
        correlations = [] 

        # All'interno di ogni clip, itero su ogni singolo neurone (da 0 a 226)
        for neuron in range(y_true_example.shape[1]):
            # Estrae la serie temporale (66 frame) per UN singolo neurone
            true_vals = y_true_example[:, neuron]
            pred_vals = y_pred_example[:, neuron]
            
            # Controlla che ci sia varianza (non una linea piatta) per evitare errori  
            true_var = np.var(true_vals)
            pred_var = np.var(pred_vals)
            
            if true_var > 1e-10 and pred_var > 1e-10:
                # Calcola la Correlazione di Pearson tra la predizione e la realtà
                corr = np.corrcoef(true_vals, pred_vals)[0, 1]

                # Aggiunge solo se il risultato è un numero valido
                if not np.isnan(corr) and np.isfinite(corr):
                    correlations.append(corr)
        
        # Calcola la correlazione media di tutti i neuroni per questo campione
        mean_corr_example = np.mean(correlations) if correlations else 0.0
        example_correlations.append(mean_corr_example) 
    
    # Calcola la media di tutti i campioni nel batch
    overall_mean = np.mean(example_correlations) if example_correlations else 0.0
    
    return {
        'eval_average_single_trial_correlation': overall_mean,
        'eval_single_trial_std': np.std(example_correlations) if example_correlations else 0.0,
        'eval_num_examples': len(example_correlations),
    }

# Strategia di Training 1: Head-Only (Shifter & Readout)

**Poiché stiamo lavorando su un nuovo set di neuroni, i pesi dei readouts del modello pre-addestrato non sono compatibili**

**Il codice esegue un caricamento parziale: estrae e carica solo i pesi del core (filtrando tramite la stringa "core.")** 

**e lascia che le nuove head (shifter e readout) vengano inizializzate da zero.**

**L'argomento strict=False permette di caricare i pesi anche se il dizionario non corrisponde perfettamente a tutti i parametri del modello.**


In [ ]:
# Istanzia il modello Viv1T con le coordinate dei neuroni
viv1t = Model(args, neuron_coordinates=neuron_coordinates)
# Carica il checkpoint del modello dal percorso specificato
checkpoint = torch.load("/kaggle/input/viv1t/transformers/default/1/model_state.pt", map_location=args.device, weights_only=False)
# Estrae il dizionario degli stati del modello
state_dict = checkpoint['model']
# Inizializza il dizionario filtrato per i pesi del core
filtered_checkpoint = {}
# Filtra soltanto i parametri che appartengono al modulo 'core.'
for key, value in state_dict.items():
    if key.startswith('core.'):
        filtered_checkpoint[key] = value
# Carica i pesi filtrati nel modello
viv1t.load_state_dict(filtered_checkpoint, strict=False)
# Sposta il modello sulla gpu
viv1t = viv1t.to(args.device)

## Congelamento dei Parametri del Core

In [ ]:
# Congela tutti i parametri del modello (li rende non addestrabili)
for param in viv1t.parameters():
    param.requires_grad = False

# Sblocca i parametri del modulo shifters (li rende addestrabili)
for param in viv1t.shifters.parameters():
    param.requires_grad = True

# Sblocca i parametri del modulo readouts (li rende addestrabili)
for param in viv1t.readouts.parameters():
    param.requires_grad = True

## Configurazione e Avvio del Training

Questa sezione definisce gli iperparametri e le strategie di addestramento per la fase Head-Only (Shifter + Readout):
- `num_train_epochs`: numero di volte che il modello vede i dati di training ==> aggiorna i pesi                           dopo ogni batch sia in fase di Learning sia in fase di Inferenza
- `per_device_train_batch_size` / `per_device_eval_batch_size`: processiamo un solo video alla volta  a causa della memoria limitata di kaggle
- `learning_rate`: velocità di apprendimento iniziale (1e-3) per la testa del modello.
- `weight_decay`: Penalizza i pesi del modello se diventano troppo grandi, costringendo il modello a                   imparare feature più semplici ==> riduce overfitting.
- `fp16`: (Mixed Precision) Invece di usare numeri a 32 bit (float32), usa numeri a 16 bit dove possibile
- `lr_scheduler_type`: learning rate non rimane fissa a 1e-3. Segue una curva a coseno
- `eval_strategy`: Alla fine di ogni epoca il modello si ferma e fa il test sul Validation Set.
- `do_eval`: abilita il ciclo di validazione durante l'addestramento
- `save_strategy`: salva solo il miglior checkpoint secondo la metrica scelta.
- `save_total_limit`: limita il numero di checkpoint conservati (1).
- `output_dir`: cartella dove vengono salvati artefatti e checkpoint dell’esperimento.
- `metric_for_best_model`: metrica usata per selezionare il checkpoint ottimale (`eval_average_single_trial_correlation`).
- `greater_is_better`: indica che valori più alti della metrica sono migliori.
- `load_best_model_at_end`: ricarica il miglior checkpoint al termine del training per valutazioni finali.
- `logging_steps`: ogni 10 step registra metriche e log (invio a backend).
- `report_to`: sistema di logging esterno (wandb) per tracking esperimento.
- `push_to_hub`: disabilita upload automatico su Hugging Face Hub durante il training.
- `hub_strategy`: strategia di upload (qui non usata perché `push_to_hub=False`).

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=40, #30
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=1e-3, 
    weight_decay=0.01,# regolarizzazione L2
    fp16=True,
    lr_scheduler_type="cosine",

    # Validazione
    eval_strategy="epoch", 
    do_eval=True, # abilita esecuzione automatica di compute_metrics sul validation set
    save_strategy="best",
    save_total_limit=1,

    # Selezione del modello
    output_dir="./results/Head-40epochs",
    metric_for_best_model="eval_average_single_trial_correlation", 
    greater_is_better=True,  
    load_best_model_at_end=True,  
    
    # Logging e Upload
    logging_steps=10, # ogni 10 step salva metriche e log
    report_to="wandb",

    # Upload su Hugging Face Hub
    push_to_hub=False, 
    hub_strategy="every_save",
    hub_token=hf_hub_token
)

In [ ]:
viv1t_wrapped = ViViTTrainerWrapper(viv1t, mouse_id='A')

trainer_viv1t_wrapped = CustomViViTTrainer(
    model=viv1t_wrapped,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=validation_set,
    compute_metrics=compute_metrics,
)

**Inizializzazione del tracciamento esperimento (Weights & Biases)**
Avvia una run `wandb` per registrare iperparametri e metadata del training del modello ViViT (solo moduli Shifter e Readout addestrabili)

In [ ]:
wandb.init(
    project="Brain-Encoding-ViV1T-Trainings",
    entity="c-h-r-o-ll-o16198-8-universit-catania",
    name="ViViT_Shifter_Readout_Only-V1",
    config={
        # Parametri di addestramento standard
        "learning_rate": training_args.learning_rate,  
        "epochs": training_args.num_train_epochs,
        "batch_size": training_args.per_device_train_batch_size,
        "lr_scheduler_type": training_args.lr_scheduler_type,
        "warmup_steps": training_args.warmup_steps,           
        "weight_decay": training_args.weight_decay,           
        "max_grad_norm": training_args.max_grad_norm,          
        
        # Informazioni sul modello
        "model_architecture": "ViViT_Shifter_Readout_Only",
        "optimizer_type": "AdamW_custom_lr",
        "gradient_accumulation_steps": training_args.gradient_accumulation_steps,
        "fp16": training_args.fp16,
        "save_strategy": training_args.save_strategy,
    }
)

In [ ]:
trainer_viv1t_wrapped.train()

In [ ]:
wandb.finish()

In [ ]:
# miglior valore della metrica di monitoraggio registrato durante l'intero ciclo di addestramento.
trainer_viv1t_wrapped.state.best_metric

## Salvataggio (Head-Only)

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

api.create_repo(
    repo_id="robyrava/vivit-brain-encoding",
    token=hf_hub_token,
    private=False,  
    exist_ok=True  
)

api.upload_folder(
    folder_path="./results/Head-40epochs/",
    repo_id="robyrava/vivit-brain-encoding",
    path_in_repo="experiments/Head-40epochs/",
    token=hf_hub_token
)

**CARICAMENTO DA HUGGINGFACE**

In [ ]:
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file


our_checkpoint_path = hf_hub_download(
    repo_id="robyrava/vivit-brain-encoding",
    filename="experiments/Head-40epochs/checkpoint-5642/model.safetensors",  
    token=hf_hub_token
)

state_dict = load_file(our_checkpoint_path)

viv1t = Model(args, neuron_coordinates=neuron_coordinates)
viv1t = viv1t.to(args.device)
viv1t_wrapped = ViViTTrainerWrapper(viv1t, mouse_id='A')

viv1t_wrapped.load_state_dict(state_dict)

trainer_viv1t_wrapped = CustomViViTTrainer(
    model=viv1t_wrapped,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=validation_set,
    compute_metrics=compute_metrics,
)

# Controllo rapido (sanity check) sulla validazione
validation_results = trainer_viv1t_wrapped.evaluate(eval_dataset=validation_set)
print(f"Risultati (eval_single_trial_correlation) sul validation set: {validation_results['eval_average_single_trial_correlation']}")

# Valutazione del modello sul test set
test_results = trainer_viv1t_wrapped.evaluate(eval_dataset=test_set)
print(f"Risultati (eval_single_trial_correlation) sul test set: {test_results['eval_average_single_trial_correlation']}")

**Commento Risultati (Head-Only)**

**Validazione:** correlazione ≈ 0.026, leggermente sopra la baseline (≈0.0235); il modello coglie una minima struttura del segnale neurale sui trial di validazione.
**Test:** correlazione ≈ -0.00077 (≈0), non generalizza; indica overfitting o scarsa robustezza del mapping neurale finale.
**Sintesi:** la testa (shifter + readout) apprende pattern specifici del set di validazione ma non trasferisce informazione ai trial non visti.


# Strategia di Training 2: LoRA + Head

In [ ]:
CURRENT_RANK = 8  #8, 16, o 32
TRAIN_EPOCHS = 40

**target_modules**: dove inserire le matrici LoRA all'interno del Transformer.

* fused_linear: gestisce le proiezioni Query, Key, Value dell'Attenzione. È il posto standard per LoRA.

* attn_out: La proiezione in uscita dal blocco di attenzione.

* ff_out.2: La parte finale del blocco Feed-Forward (MLP).

In [ ]:
import torch
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainingArguments
from huggingface_hub import HfApi
import wandb
import os

LORA_CONFIGS = {
    8: {
        "alpha": 16,
        #"target_modules": ["fused_linear"],
        "target_modules": ["fused_linear", "attn_out", "ff_out.2"],
        #"learning_rate": 0.001,
        "learning_rate": 0.0036, # molto più alto perchè con r=8 ci sono pochi parametri da addestrare
        #"scheduler": "cosine",
        "scheduler": "linear", # più adatto a learning_rate alti
        #"description": "fused_only_cosine"
        "description": "all_layers_linear"
    },
    16: {
        "alpha": 32,
        #"target_modules": ["fused_linear", "attn_out", "ff_out.2"], # "All" layers
        "target_modules": ["fused_linear"],
        "learning_rate": 0.001,
        "scheduler": "linear",
        #"description": "all_layers_linear"
        "description": "fused_linear"
    },
    32: {
        "alpha": 64,
        #"target_modules": ["fused_linear", "attn_out", "ff_out.2"], # "All" layers
        "target_modules": ["fused_linear"], # Solo "Fused"
        #"learning_rate": 0.0036,
        "learning_rate": 0.001,
        #"scheduler": "linear",
        "scheduler": "cosine",
        #"description": "all_layers_linear_highLR"
        "description": "fused_only_cosine"
    }
}

# Recupero della configurazione corrente
current_config = LORA_CONFIGS[CURRENT_RANK]


run_name = f"ViViT_LoRA-{CURRENT_RANK}_{current_config['description']}"
output_dir = f"./results/{run_name}"
repo_path = f"experiments/{run_name}"

print(f"--- Starting Experiment: Rank {CURRENT_RANK} ---")
print(f"Configuration: {current_config}")

## PREPARAZIONE DEL MODELLO E LORA

In [ ]:
peft_config = LoraConfig(
    r=CURRENT_RANK,
    lora_alpha=current_config["alpha"],
    lora_dropout=0.1, #durante l'addestramento, spegne casualmente il 10% dei neuroni nelle matrici LoRA ad ogni passaggio.
    bias="none", #alleniamo solo i pesi (w) ignorando i bias
    task_type=TaskType.FEATURE_EXTRACTION, #perchè è un modello che prende un input (video) e ne estrae una rappresentazione numerica (le feature) che poi userò per predire l'attività neurale
    target_modules=current_config["target_modules"] #quali layer del modello originale devono essere "avvolti" dalle matrici LoRA
)

In [ ]:
# Caricamento del Modello Base
viv1t = Model(args, neuron_coordinates=neuron_coordinates)

# Caricamento pesi pre-addestrati (solo core)
checkpoint = torch.load("/kaggle/input/viv1t/transformers/default/1/model_state.pt", map_location=args.device, weights_only=False)
state_dict = checkpoint['model']

filtered_checkpoint = {}
for key, value in state_dict.items():
    if key.startswith('core.'):
        filtered_checkpoint[key] = value

viv1t.load_state_dict(filtered_checkpoint, strict=False)
viv1t = viv1t.to(args.device)

In [ ]:
# Applicazione di LoRA
lora_model = get_peft_model(ViV1TWrapper(viv1t), peft_config) 
lora_model.print_trainable_parameters()


In [ ]:
# Scongelamento delle head (Shifter & Readout)
# È necessario rendere trainabili anche le parti non-LoRA specifiche per il topo 'A'
for param in lora_model.base_model.model.vivit_model.shifters.parameters():
    param.requires_grad = True
    
for param in lora_model.base_model.model.vivit_model.readouts.parameters():
    param.requires_grad = True
    
print("Trainable parameters after unfreezing heads:")
lora_model.print_trainable_parameters()

## TRAINING SETUP

In [ ]:
# Argomenti di addestramento
training_args = TrainingArguments(
    num_train_epochs=TRAIN_EPOCHS,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    
    # Parametri dinamici dal dizionario
    learning_rate=current_config["learning_rate"], 
    lr_scheduler_type=current_config["scheduler"],
    
    weight_decay=0.01,
    fp16=True,

    # Validazione e salvataggio
    eval_strategy="epoch",
    save_total_limit=1,
    do_eval=True, 
    save_strategy="best",
    
    # Selezione del modello
    output_dir=output_dir,
    metric_for_best_model="eval_average_single_trial_correlation", 
    greater_is_better=True,  # Ci importa che la Correlazione sia alta
    load_best_model_at_end=True,  
    
    # Logging
    logging_steps=10, 
    report_to="wandb",
    run_name=run_name, 

    # Hub
    push_to_hub=False,  
    hub_strategy="every_save",
    hub_token=hf_hub_token
)

In [ ]:
# Inizializzazione Trainer
lora_viv1t_wrapped = ViViTTrainerWrapper(lora_model, mouse_id='A')

trainer = CustomViViTTrainer(
    model=lora_viv1t_wrapped,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=validation_set,
    compute_metrics=compute_metrics,
)

In [ ]:
# Inizializzazione WandB
wandb.init(
    project="Brain-Encoding-ViV1T-Trainings",
    entity="c-h-r-o-ll-o16198-8-universit-catania",
    name=run_name,
    config={
        "rank": CURRENT_RANK,
        "alpha": current_config["alpha"],
        "target_modules": current_config["target_modules"],
        "learning_rate": current_config["learning_rate"],
        "scheduler": current_config["scheduler"],
        "epochs": TRAIN_EPOCHS,
        "model_architecture": f"ViViT_LoRA-{CURRENT_RANK}",
    }
)

## ESECUZIONE

In [ ]:
# Avvio Training
trainer.train()

In [ ]:
# Upload su Hugging Face Hub
api = HfApi()
api.create_repo(
    repo_id="robyrava/vivit-brain-encoding",
    token=hf_hub_token,
    private=False,  
    exist_ok=True  
)

print(f"Uploading results to {repo_path}...")
api.upload_folder(
    folder_path=output_dir, 
    repo_id="robyrava/vivit-brain-encoding",
    path_in_repo=repo_path, 
    token=hf_hub_token
)

print(f"Best Validation Metric: {trainer.state.best_metric}")

In [ ]:
# Valutazione Finale (usa il modello 'best' caricato automaticamente alla fine del training)
val_results = trainer.evaluate(eval_dataset=validation_set)
print(f"Final Validation Score: {val_results['eval_average_single_trial_correlation']}")

test_results = trainer.evaluate(eval_dataset=test_set)
print(f"Final Test Score: {test_results['eval_average_single_trial_correlation']}")

In [ ]:
wandb.finish()

# CONFRONTO RISULTATI CON LA BASELINE

In [ ]:
baseline_result = 0.0235345645802377

In [ ]:
def print_result(test_results):
    ratio = test_results/baseline_result
    if ratio < 1:
        print("Il modello riesce a spiegare meno dei dati stessi")
    elif ratio > 1:
        print(f"Il modello sta spiegando non solo la media, ma anche qualcosa in più (migliore di {ratio:.2f}) volte")
    else:
        print("Sono uguali")

## Risultati Head-Only

In [ ]:
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

our_checkpoint_path = hf_hub_download(
    repo_id="robyrava/vivit-brain-encoding",
    filename="experiments/Head-V1/checkpoint-217/model.safetensors",
    token=hf_hub_token
)

state_dict = load_file(our_checkpoint_path)

viv1t = Model(args, neuron_coordinates=neuron_coordinates)
viv1t = viv1t.to(args.device)
viv1t_wrapped = ViViTTrainerWrapper(viv1t, mouse_id='A')

viv1t_wrapped.load_state_dict(state_dict)


trainer_viv1t_wrapped = CustomViViTTrainer(
    model=viv1t_wrapped,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=validation_set,
    compute_metrics=compute_metrics,
)

test_results = trainer_viv1t_wrapped.evaluate(eval_dataset=test_set)['eval_average_single_trial_correlation']
print(f"Risultati sul test set: {test_results}")
print(f"Risulato baseline: {baseline_result}")

print_result(test_results)

## Risultati LoRA (r=8, 16, 32)

In [ ]:
#vedere su huggingface
checkpoint_filename = "experiments/ViViT_LoRA-8_all_layers_linear/checkpoint-8029/model.safetensors" 
NUM_CHECKPOINT = 8029

In [ ]:
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

print(f"Tentativo di download del file: {checkpoint_filename}")

#Ricaricamento dello Stato del Modello
try:
    our_checkpoint_path = hf_hub_download(
        repo_id="robyrava/vivit-brain-encoding",
        filename=checkpoint_filename,
        token=hf_hub_token
    )

    state_dict = load_file(our_checkpoint_path)

    # Re-inizializzazione del Modello Base (Senza PEFT)
    viv1t = Model(args, neuron_coordinates=neuron_coordinates)
    
    # Re-inizializzazione della configurazione LoRA
    lora_config = LoraConfig(
        r=CURRENT_RANK,
        lora_alpha=current_config["alpha"],
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.FEATURE_EXTRACTION,
        target_modules=current_config["target_modules"]
    )
    
    # Re-applicazione di LoRA al modello
    lora_model = get_peft_model(ViV1TWrapper(viv1t), lora_config) 
    
    # Il caricamento dei pesi deve essere fatto sul modello wrapped
    lora_viv1t_wrapped = ViViTTrainerWrapper(lora_model, mouse_id='A')
    lora_viv1t_wrapped.load_state_dict(state_dict)

# Valutazione

    # Il trainer va re-inizializzato con il modello ricaricato e i training_args
    # Riusa training_args e compute_metrics definiti nelle celle precedenti
    trainer_lora_viv1t_wrapped = CustomViViTTrainer(
        model=lora_viv1t_wrapped,
        args=training_args, 
        train_dataset=train_set,
        eval_dataset=validation_set,
        compute_metrics=compute_metrics,
    )

    # Valutazione del modello sui dati di test
    test_results = trainer_lora_viv1t_wrapped.evaluate(eval_dataset=test_set)['eval_average_single_trial_correlation']
    
    print(f"\n--- Risultati LoRA Rank {CURRENT_RANK} (Checkpoint Ricaricato: {NUM_CHECKPOINT}) ---")
    print(f"Risultati (eval_single_trial_correlation) sul test set: {test_results}")
    
    print(f"Risultato baseline: {baseline_result}")
    print_result(test_results)

except Exception as e:
    print(f"\nERRORE nel caricamento del checkpoint {NUM_CHECKPOINT}: {e}")
    print("Controlla che il numero del checkpoint e la configurazione del Rank (CURRENT_RANK) siano corretti.")